# 🔬 VQE Ground State Simulation of H₂ with Ansatz Comparison (Noiseless)

This notebook implements the **Variational Quantum Eigensolver (VQE)** to compute the **ground state energy of dihydrogen (H₂)** using a **noiseless quantum simulator**. It compares the performance of multiple **variational ansatzes** to understand their expressive power and convergence behavior.

### Key Features:
- **Molecular Setup**: Defines the H₂ molecule and constructs the qubit-based molecular Hamiltonian using the STO-3G basis.
- **Customizable Ansatz Selection**:
  - Uses a dictionary of predefined ansatzes (`ANSATZES`) including:
    - `TwoQubit-RY-CNOT` (toy 2-qubit entangler)
    - `Minimal` (simplified configuration)
    - `RY-CZ` (hardware-efficient style)
- **VQE Execution**:
  - Runs independent VQE loops for each ansatz using the **Adam optimizer**.
  - Tracks energy at each iteration for performance comparison.

### 📊 Visualization Outputs:
- **Energy convergence plots** for each ansatz over the training iterations.
- Final ground state energies per ansatz, enabling performance benchmarking.

### Purpose:
To evaluate and compare different **ansatz strategies** for variational quantum simulations, providing insight into trade-offs between expressivity, convergence speed, and implementation complexity in near-term quantum devices.

In [ ]:
import pennylane as qml                  # Quantum circuit builder and device management
from pennylane import numpy as np        # Not regular NumPy, but similar and supports automatic differentiation
from pennylane import qchem              # Quantum chemistry module used to define molecule Hamiltonians
import matplotlib.pyplot as plt
import os
from vqe_utils import ANSATZES, get_optimizer, init_params, set_seed  # helpers

set_seed(0)                              # Reproducible params for ansatz init
IMG_DIR = "images"                       # Single image directory used repo-wide
os.makedirs(IMG_DIR, exist_ok=True)

In [ ]:
# Define the atoms in the dihydrogen molecule
symbols = ["H", "H"]

# Define the coordinates (in Ångströms)
coordinates = np.array([
    [0.0000, 0.0000, 0.0000],  # Hydrogen 1
    [0.0000, 0.0000, 0.7414]   # Hydrogen 2
])

# Define the dihydrogen Hamiltonian and the number of qubits required
# Default STO-3G basis set
hamiltonian, qubits = qchem.molecular_hamiltonian(symbols, coordinates, charge=0)

# Define the number of required quantum wires / logical qubits
num_wires = qubits
print(f"{qubits} qubits required")

# Create quantum device simulator backend
dev = qml.device("default.qubit",  # Noiseless state vector simulator
                 wires=num_wires)


def create_circuit(ansatz_fn, wires):
    @qml.qnode(dev)
    def circuit(params):
        ansatz_fn(params, wires=wires)
        return qml.expval(hamiltonian)
    return circuit

In [ ]:
# Run VQE for each ansatz
results = {}
max_iters = 40
stepsize = 0.2

for name, ansatz_fn in ANSATZES.items():
    print(f"\nRunning VQE with {name} Ansatz...")

    circuit = create_circuit(ansatz_fn, wires=[0, 1])
    cost = lambda p: circuit(p)

    # Consistent initialization via helper (keeps same shapes as before)
    params = init_params(name, num_wires=2, scale=0.01, requires_grad=True)

    opt = get_optimizer("Adam", stepsize=stepsize)

    energy_progress = []
    for i in range(max_iters):
        params = opt.step(cost, params)
        energy = cost(params)
        energy_progress.append(energy)

    results[name] = {
        "final_energy": energy,
        "params": params,
        "trajectory": energy_progress,
    }

    print(f"Final energy for {name}: {energy:.8f} Ha")

In [ ]:
plt.figure(figsize=(8, 5))
for name, data in results.items():
    plt.plot(data["trajectory"], label=name)

plt.xlabel("Iteration")
plt.ylabel("Energy (Ha)")
plt.title("H₂ VQE: Ansatz Comparison (Noiseless)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig(f"{IMG_DIR}/H2_Ansatz_Comparison.png", dpi=300)
plt.show()

In [ ]:
for name, ansatz_fn in ANSATZES.items():
    print(f"\n{name} Ansatz Circuit for H₂:")
    params = results[name]["params"]

    fig, ax = qml.draw_mpl(ansatz_fn)(params, wires=[0, 1])
    fig.suptitle(f"{name} Ansatz", fontsize=14)
    fig.savefig(f"{IMG_DIR}/H2_{name}_Circuit.png", dpi=300, bbox_inches="tight")
    plt.show()